<a href="https://colab.research.google.com/github/vishnuvryeruva/Market-Basket-Item-Apriori-Algorithm-Dataset-from-real-world-retailer-/blob/main/Market_Basket.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install apyori

  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5974 sha256=eaafc4ad27f444e815cf50d1aadc0affac1f9c2b554736411231f30760ab5b55
  Stored in directory: /root/.cache/pip/wheels/cb/f6/e1/57973c631d27efd1a2f375bd6a83b2a616c4021f24aab84080
Successfully built apyori


In [2]:
from apyori import apriori, load_transactions
import pandas as pd
import numpy as np

In [3]:
def predict(dataset, test, num_items):
  length = np.array([len(i) for i in dataset["items"]])
  items = set()
  for item in dataset[length == num_items]["items"]:
    if all([True if i in item else False for i in test]) :
      items.add(item)
  return items

In [6]:
min_support = 0.0045
train_dataset = pd.read_csv("/content/drive/MyDrive/Market Basket/TRAIN-ARULES.csv")
test_dataset = pd.read_csv("/content/drive/MyDrive/Market Basket/testarules.csv", keep_default_na = False).values
num_test_items = [len(item) for item in test_dataset]
test_dataset = [[" ".join(i.split(",")) for i in j if i != ""] for j in test_dataset]
train_dataset.product_name = ["".join(i.split(",")) for i in train_dataset.product_name]

In [7]:
train_dataset

,order_id,user_id,product_name
0,1483,90,Organic Pink Lemonade Bunny Fruit Snacks
1,1483,90,Dark Chocolate Minis
2,1483,90,Sparkling Water Natural Mango Essenced
3,1483,90,Peach-Pear Sparkling Water
4,1483,90,Organic Heritage Flakes Cereal
...,...,...,...
12958,3413579,84,Organic Avocado
12959,3413579,84,Guacamole
12960,3413579,84,Thin And Crispy Corn Tortilla Chips
12961,3413579,84,Original Veggie Straws


In [8]:
test_dataset

[['Dark Chocolate Minis',
  'Organic Pink Lemonade Bunny Fruit Snacks',
  'Peach-Pear Sparkling Water']]

In [10]:
train_dataset.shape

(12963, 3)

In [11]:
train_dataset[train_dataset.order_id == 1483].shape

(12, 3)

In [12]:
train_dataset[train_dataset.user_id == 90].shape

(670, 3)

In [13]:
order_labels = train_dataset.order_id.unique()
transactions = [train_dataset[train_dataset.order_id == i]["product_name"].tolist() for i in order_labels]

In [14]:
train = apriori(transactions = transactions, min_support = min_support)
apriori_result = list(train)
apriori_result = pd.DataFrame(apriori_result)
apriori_result["confidence"] = 0.0
apriori_result["lift"] = 0.0
for i in range(apriori_result.shape[0]):
  apriori_result["confidence"][i] = apriori_result.ordered_statistics[i][0].confidence
  apriori_result["lift"][i] = apriori_result.ordered_statistics[i][0].lift
  apriori_result["items"][i] = tuple(apriori_result["items"][i])
apriori_result = apriori_result.drop("ordered_statistics", axis = 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [15]:
apriori_result.head()

,items,support,confidence,lift
0,"(0% Greek Strained Yogurt,)",0.009873,0.009873,1.0
1,"(100% Juice Variety Pack,)",0.004937,0.004937,1.0
2,(100% Premium Select Not From Concentrate Pure...,0.009168,0.009168,1.0
3,"(100% Recycled Paper Towels,)",0.005642,0.005642,1.0
4,"(1500 Pale Ale,)",0.014810,0.014810,1.0


In [16]:
result = {}
for i, (test_row, num_test_row) in enumerate(zip(test_dataset, num_test_items)):
  result["row " + str(i)] = predict(apriori_result, test_row, num_test_row)

In [17]:
test_dataset

[['Dark Chocolate Minis',
  'Organic Pink Lemonade Bunny Fruit Snacks',
  'Peach-Pear Sparkling Water']]

In [18]:
for i,item in enumerate(result.keys()):
  print("Basket items prediction for transaction:-\n\n Transaction",i+1,": {"," ,".join(test_dataset[i]),"}\n")
  for j,pred in enumerate(result[item]):
    print(str(j+1)+". {"+", ".join(set(pred).difference(test_dataset[i])),"}")
  print("\n")

Basket items prediction for transaction:-

 Transaction 1 : { Dark Chocolate Minis ,Organic Pink Lemonade Bunny Fruit Snacks ,Peach-Pear Sparkling Water }

1. {Maple Pumpkin Seeds with Sea Salt Chewy with a Crunch Granola Bars, Organic Heritage Flakes Cereal }
2. {Sparkling Water Natural Mango Essenced, Organic Heritage Flakes Cereal }
3. {Sparkling Water Natural Mango Essenced, Organic Graham Crunch Cereal }
4. {Organic Heritage Flakes Cereal, Organic Graham Crunch Cereal }


